In [1]:
from datasets import load_dataset
from datasets import Dataset

#### Load Dataset

https://huggingface.co/datasets/databricks/databricks-dolly-15k

In [2]:
dataset_name = "databricks/databricks-dolly-15k"

dataset = load_dataset(dataset_name, split="train[0:2048]")

print(f"Loaded {len(dataset)} examples")
print("\nFirst example:")

print(f"Instruction: {dataset['instruction'][0]}")
print(f"Response: {dataset['response'][0]}")

print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Loaded 2048 examples

First example:
Instruction: When did Virgin Australia start operating?
Response: Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.
Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 2048
})


#### Apply Formatting

In [3]:
def prompt_fn(data):

  """
  Returns a list of formatted string
  Only keeps open_qa, closed_qa, general_qa
  """

  formatted_text = []

  instruction = data['instruction']
  context = data['context']
  response = data['response']
  category = data['category']

  for i in range(len(instruction)):
    if category[i] in ["open_qa", "closed_qa", "general_qa"]:

      if context[i]:
        text = f"### Instruction:{instruction[i]}\n\n### Context:{context[i]}\n\n### Response:{response[i]}"
      else:
        text = f"### Instruction:{instruction[i]}\n\n### Response:{response[i]}"
      formatted_text.append(text)

  return formatted_text

texts = prompt_fn(dataset)

train_dataset = Dataset.from_dict({"text": texts})

In [4]:
print(train_dataset)
print("\nSample formatted example:")
print(train_dataset[0]["text"])

Dataset({
    features: ['text'],
    num_rows: 1092
})

Sample formatted example:
### Instruction:When did Virgin Australia start operating?

### Context:Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.

### Response:Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.


#### Save as JSONL and upload it to Huggingface Hub

In [5]:
!pip install jsonlines

In [6]:
import jsonlines

output_file = "dolly-instruct-1k.jsonl"

with jsonlines.open(output_file, 'w') as writer:
  writer.write_all(train_dataset)